# Mxnet BYOM: Train locally and deploy on SageMaker.

1. [Introduction](#Introduction)
2. [Prerequisites and Preprocessing](#Prequisites-and-Preprocessing)
    1. [Permissions and environment variables](#Permissions-and-environment-variables)
    2. [Data Setup](#Data-setup)
3. [Training the network locally](#Training)
4. [Set up hosting for the model](#Set-up-hosting-for-the-model)
    1. [Export from MXNet](#Export-the-model-from-mxnet)
    2. [Import model into SageMaker](#Import-model-into-SageMaker)
    3. [Create endpoint](#Create-endpoint) 
5. [Validate the endpoint for use](#Validate-the-endpoint-for-use)


__Note__: Compare this with the [tensorflow bring your own model example](../tensorflow_iris_byom/tensorflow_BYOM_iris.ipynb)

## Introduction
In this notebook, we will train a neural network locally on the location from where this notebook is run using MXNet. We will then see how to create an endpoint from the trained MXNet model and deploy it on SageMaker. We will then inference from the newly created SageMaker endpoint. 

The neural network that we will use is a simple fully-connected neural network. The definition of the neural network can be found in the accompanying [mnist.py](mnist.py) file. The ``build_graph`` method contains the model definition (shown below).

```python
def build_graph():
    data = mx.sym.var('data')
    data = mx.sym.flatten(data=data)
    fc1 = mx.sym.FullyConnected(data=data, num_hidden=128)
    act1 = mx.sym.Activation(data=fc1, act_type="relu")
    fc2 = mx.sym.FullyConnected(data=act1, num_hidden=64)
    act2 = mx.sym.Activation(data=fc2, act_type="relu")
    fc3 = mx.sym.FullyConnected(data=act2, num_hidden=10)
    return mx.sym.SoftmaxOutput(data=fc3, name='softmax')
```

From this definitnion we can see that there are two fully-connected layers of 128 and 64 neurons each. The activations of the last fully-connected layer is then fed into a Softmax layer of 10 neurons. We use 10 neurons here because the datatset on which we are going to predict is the MNIST dataset of hand-written digit recognition which has 10 classes. More details can be found about the dataset on the [creator's webpage](http://yann.lecun.com/exdb/mnist/).

## Prequisites and Preprocessing

### Permissions and environment variables

Here we set up the linkage and authentication to AWS services. In this notebook we only need the roles used to give learning and hosting access to your data. The Sagemaker SDK will use S3 defualt buckets when needed. Supply the role in the variable below.

In [1]:
import boto3, re
from sagemaker import get_execution_role

role = get_execution_role()

### Data setup

Next, we need to pull the data from the author's site to our local box. Since we have ``mxnet`` utilities, we will use the utilities to download the dataset locally.

In [2]:
import mxnet as mx

data = mx.test_utils.get_mnist()

### Training

It is time to train the network. Since we are training the network locally, we can make use of mxnet training tools. The training method is also in the accompanying [mnist.py](mnist.py) file. The method is as follows. 

```python 
def train(data, hyperparameters= {'learning_rate': 0.11}, num_cpus=1, num_gpus =0 , **kwargs):
    train_labels = data['train_label']
    train_images = data['train_data']
    test_labels = data['test_label']
    test_images = data['test_data']
    batch_size = 100
    train_iter = mx.io.NDArrayIter(train_images, train_labels, batch_size, shuffle=True)
    val_iter = mx.io.NDArrayIter(test_images, test_labels, batch_size)
    logging.getLogger().setLevel(logging.DEBUG)
    mlp_model = mx.mod.Module(
        symbol=build_graph(),
        context=get_train_context(num_cpus, num_gpus))
    mlp_model.fit(train_iter,
                  eval_data=val_iter,
                  optimizer='sgd',
                  optimizer_params={'learning_rate': float(hyperparameters.get("learning_rate", 0.1))},
                  eval_metric='acc',
                  batch_end_callback=mx.callback.Speedometer(batch_size, 100),
                  num_epoch=10)
    return mlp_model
```

The method above collects the ``data`` variable that ``get_mnist`` method gives you (which is a dictionary of data arrays) along with a dictionary of ``hyperparameters`` which only contains learning rate, and other parameters. It creates a [``mxnet.mod.Module``](https://mxnet.incubator.apache.org/api/python/module.html) from the network graph we built in the ``build_graph`` method and trains the network using the ``mxnet.mod.Module.fit`` method. 

In [3]:
from mnist import train

model = train(data=data, num_cpus=1, num_gpus=0)

INFO:root:Epoch[0] Batch [0-100]	Speed: 88262.86 samples/sec	accuracy=0.110099
INFO:root:Epoch[0] Batch [100-200]	Speed: 87784.23 samples/sec	accuracy=0.116000
INFO:root:Epoch[0] Batch [200-300]	Speed: 87146.04 samples/sec	accuracy=0.111300
INFO:root:Epoch[0] Batch [300-400]	Speed: 87671.20 samples/sec	accuracy=0.111700
INFO:root:Epoch[0] Batch [400-500]	Speed: 86419.02 samples/sec	accuracy=0.157800
INFO:root:Epoch[0] Train-accuracy=0.160133
INFO:root:Epoch[0] Time cost=0.756
INFO:root:Epoch[0] Validation-accuracy=0.495200
INFO:root:Epoch[1] Batch [0-100]	Speed: 87714.28 samples/sec	accuracy=0.592673
INFO:root:Epoch[1] Batch [100-200]	Speed: 86792.59 samples/sec	accuracy=0.756300
INFO:root:Epoch[1] Batch [200-300]	Speed: 87943.81 samples/sec	accuracy=0.803100
INFO:root:Epoch[1] Batch [300-400]	Speed: 87771.55 samples/sec	accuracy=0.819000
INFO:root:Epoch[1] Batch [400-500]	Speed: 88138.78 samples/sec	accuracy=0.842500
INFO:root:Epoch[1] Train-accuracy=0.779067
INFO:root:Epoch[1] Time c

## Set up hosting for the model

### Export the model from mxnet

In order to set up hosting, we have to import the model from training to hosting. We will begin by exporting the model from MXNet and saving it down. Analogous to the [TensorFlow example](../tensorflow_iris_byom/tensorflow_BYOM_iris.ipynb), some structure needs to be followed. The exported model has to be converted into a form that is readable by ``sagemaker.mxnet.model.MXNetModel``. The following code describes exporting the model in a form that does the same:

In [4]:
import os
import json
import tarfile

os.mkdir("model")

model.save_checkpoint("model/model", 0000)
with open("model/model-shapes.json", "w") as shapes:
    json.dump([{"shape": model.data_shapes[0][1], "name": "data"}], shapes)

def flatten(tarinfo):
    tarinfo.name = os.path.basename(tarinfo.name)
    return tarinfo


tar = tarfile.open("model.tar.gz", "w:gz")
tar.add("model", filter=flatten)
tar.close()

INFO:root:Saved checkpoint to "model/model-0000.params"


The above piece of code essentially hacks the MXNet model export into a sagemaker-readable model export. Study the exported model files if you want to organize your exports in the same fashion as well. Alternatively, you can load the model on MXNet itself and load the sagemaker model as you normally would. Refer [here](https://github.com/aws/sagemaker-python-sdk#model-loading) for details on how to load MXNet models.

### Import model into SageMaker

Open a new sagemaker session and upload the model on to the default S3 bucket. We can use the ``sagemaker.Session.upload_data`` method to do this. We need the location of where we exported the model from MXNet and where in our default bucket we want to store the model(``/model``). The default S3 bucket can be found using the ``sagemaker.Session.default_bucket`` method.

In [6]:
import sagemaker

sagemaker_session = sagemaker.Session()
inputs = sagemaker_session.upload_data(path="model.tar.gz", key_prefix="model")

DEBUG:botocore.hooks:Changing event name from creating-client-class.iot-data to creating-client-class.iot-data-plane
DEBUG:botocore.hooks:Changing event name from before-call.apigateway to before-call.api-gateway
DEBUG:botocore.hooks:Changing event name from request-created.machinelearning.Predict to request-created.machine-learning.Predict
DEBUG:botocore.hooks:Changing event name from before-parameter-build.autoscaling.CreateLaunchConfiguration to before-parameter-build.auto-scaling.CreateLaunchConfiguration
DEBUG:botocore.hooks:Changing event name from before-parameter-build.route53 to before-parameter-build.route-53
DEBUG:botocore.hooks:Changing event name from request-created.cloudsearchdomain.Search to request-created.cloudsearch-domain.Search
DEBUG:botocore.hooks:Changing event name from docs.*.autoscaling.CreateLaunchConfiguration.complete-section to docs.*.auto-scaling.CreateLaunchConfiguration.complete-section
DEBUG:botocore.hooks:Changing event name from before-parameter-buil

Use the ``sagemaker.mxnet.model.MXNetModel`` to import the model into SageMaker that can be deployed. We need the location of the S3 bucket where we have the model, the role for authentication and the entry_point where the model defintion is stored (``mnist.py``). The import call is the following:

In [9]:
from sagemaker.mxnet.model import MXNetModel

sagemaker_model = MXNetModel(
    model_data="s3://" + sagemaker_session.default_bucket() + "/model/model.tar.gz",
    role=role,
    framework_version='1.7.0',
    py_version='py3',
    entry_point="mnist.py",
)

### Create endpoint

Now the model is ready to be deployed at a SageMaker endpoint. We can use the ``sagemaker.mxnet.model.MXNetModel.deploy`` method to do this. Unless you have created or prefer other instances, we recommend using 1 ``'ml.c4.xlarge'`` instance for this training. These are supplied as arguments. 

In [10]:
import logging

logging.getLogger().setLevel(logging.WARNING)

predictor = sagemaker_model.deploy(initial_instance_count=1, instance_type="ml.m4.xlarge")

------!

### Validate the endpoint for use

You will ping the endpoint with the test images from MNIST. 

In [20]:
import gzip 
import numpy as np
import random

with gzip.open("t10k-images-idx3-ubyte.gz", "rb") as f:
    images = np.frombuffer(f.read(), np.uint8, offset=16).reshape(-1, 28, 28).astype(np.float32)

mask = random.sample(range(len(images)), 16) # randomly select some of the test images
mask = np.array(mask, dtype=np.int)

data = images[mask]

In [21]:
response = predictor.predict(data)
print("Raw prediction result:")
print(response)
print()

labeled_predictions = list(zip(range(10), response[0]))
print("Labeled predictions: ")
print(labeled_predictions)
print()

labeled_predictions.sort(key=lambda label_and_prob: 1.0 - label_and_prob[1])
print("Most likely answer: {}".format(labeled_predictions[0]))

Raw prediction result:
[[0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0], [0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0], [0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0], [0.0, 0.0, 0.0, 0.0, 2.96591231574439e-08, 0.0, 0.0, 0.0, 0.0, 1.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0], [0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0], [0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 1.0, 2.6243371788733024e-23, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]]

Labeled predictions: 
[(0, 0.0), (1, 0.0), (2, 0.0), (3, 1.0), (4, 0.0), (5, 0.0), (6, 0.0), (7, 0.0), (8, 

(Optional) Delete the Endpoint

In [26]:
sagemaker_session.delete_endpoint(
    endpoint_name=predictor.endpoint_name
)


If you do not want continied use of the endpoint, you can remove it. Remember, open endpoints are charged. If this is a simple test or practice, it is recommended to delete them.

In [ ]:
sagemaker.Session().delete_endpoint(predictor.endpoint)

Clear all stored model data so that we don't overwrite them the next time. 

In [ ]:
os.remove("model.tar.gz")
import shutil

shutil.rmtree("model")
